# <span style='background-color: #fff5b1'>오늘의 한 걸음  🚶🏽‍♂️: 마음을 나누는 챗봇</span>

## Contexts

### 1. READY
    1-1 오늘의 Exp와 Rubric  
    1-2 사용하는 라이브러리  

### 2. GAME
    2-1. 데이터 읽어오기  
    2-2. 데이터 전처리  
    2-3. 모델 학습  
        2-3-1. split
        2-3-2. model (transformer)
    2-4. 데이터 평가   

### 3. POTG
    3-1. 소감
    3-2. 어려웠던 점과 극복방안  

---


# 1. Ready
## 1-1. 오늘의 Exp와 Rubric

사실 최근에 GPT chat bot 이라는 기가막힌 챗봇이 있다.  
우리는 학습중이기 때문에 그런 성능을 내기 보다는  
전체적으로 텍스트데이터를 어떻게 다루고  
어떻게 학습시키는지를 중점적으로 다루기로 한다.

데이터셋은 AI Hub 에 있는  
wellness 대화 스크립트 데이터셋을 이용하였다.

<a href="https://aihub.or.kr/unitysearch/list.do?kwd=%EC%9B%B0%EB%8B%88%EC%8A%A4">웰니스 대화 스크립트 데이터셋</a>

## 1-2. 사용하는 라이브러리

In [4]:
import os #데이터 경로 라이브러리

import pandas as pd
import numpy as np # 수치 다루는 라이브러리


import re # 텍스트 핸들링 라이브러리

import tensorflow as tf # 텐서플로우
import tensorflow_datasets as tfds # Subword 토크나이저 라이브러리



# 2. GAME
## 2-1. 데이터 읽어오기

데이터는 두 개의 액셀 데이터를 제공하고 있다.

In [5]:
dataset_filepath = './웰니스 대화 스크립트 데이터셋/02)웰니스_대화_스크립트_데이터셋.xlsx'
df = pd.read_excel(dataset_filepath, engine='openpyxl')
df.head(1)

,핵심증상,intent,keyword(임상키워드),특이사항,연관표현,utterance,utterance(2차),response(공감),임상질문그룹(연세의료원제공),utterance(긍정),utterance(부정),긍정에 대한 챗봇 답변,부정에 대한 챗봇 답변,추가발화(190917),추가발화 (191031)
0,지속되는우울한기분(우울감),정신증상/우울감,우울,NaN,머리가 짓눌러지는 느낌/머리가 맑지 않다/침울하면서 잠도 못 자다/후회가 많고 침울...,우울해,임신해서 우울해,기분이 우울하시군요. 00님에게 스트레스 받는 일이 있었던 건 아닌지 걱정스러워요.,"거의 매일, 하루 중 대부분 시간을 우울한 기분으로 있거나 가라앉았던 적이 있나요?",응,아니,저런… (우는 이모티콘) 기분이 나아지지 않는 상태군요.,그렇군요. 기분이 계속 처지신다면 편안한 음악을 한번 들어보는 것도 좋을 것 같아요.,NaN,NaN


In [6]:
dataset_filepath = './웰니스 대화 스크립트 데이터셋/웰니스_대화_스크립트_데이터셋.xlsx'
df2 = pd.read_excel(dataset_filepath, engine='openpyxl')
df2.head()

,구분,유저,챗봇,Unnamed: 3
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,NaN
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,NaN
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",NaN
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN,NaN


위와 같이 하나는 질문과 답변, 그리고 그에 대한 세부 정보들로 이루어져 있고,  
다른 하나는 질문과 챗봇의 답변 데이터로 구성되어 있다.

In [7]:
df.columns

Index(['핵심증상', 'intent', 'keyword(임상키워드)', '특이사항', '연관표현', 'utterance',
       'utterance(2차) ', 'response(공감)', '임상질문그룹(연세의료원제공)', 'utterance(긍정)',
       'utterance(부정)', '긍정에 대한 챗봇 답변', '부정에 대한 챗봇 답변', '추가발화(190917)',
       '추가발화 (191031)'],
      dtype='object')

특히 첫번째 자료의 경우 상당한 양의 자료가 있지만, 아쉽게도 군데군데  
빠진 데이터가 많았다.

In [8]:
df.size

296535

In [9]:
df.isnull().sum()

핵심증상               19759
intent             19750
keyword(임상키워드)     19555
특이사항               19736
연관표현               19747
utterance          15452
utterance(2차)         92
response(공감)       19695
임상질문그룹(연세의료원제공)    19634
utterance(긍정)      17069
utterance(부정)      17069
긍정에 대한 챗봇 답변       19715
부정에 대한 챗봇 답변       19715
추가발화(190917)       19769
추가발화 (191031)      19769
dtype: int64

In [10]:
df.shape

(19769, 15)

그러나 총 19769개의 행 수중 다른 것들은 90%가 결측치로 존재하여  
한 쌍으로 묶어 사용하기 어렵다고 생각했다.

In [11]:
df2.shape

(5231, 4)

In [12]:
df2.isnull().sum()

구분               0
유저               0
챗봇            4197
Unnamed: 3    5231
dtype: int64

이에 반해 두번째 데이터셋은 유저와 챗봇의 쌍 데이터가  
1000종 있어 이것으로 학습을 하기로 결정했다.

In [13]:
data = df2[["유저", "챗봇"]]

In [14]:
data_filter = data.dropna()
data_filter

,유저,챗봇
0,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
15,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
16,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.
...,...,...
5196,"그 사람이 응급실 의사한테 뭐라고 속닥거리니까, 저보고 갑자기 응급처치 끝났다고, ...",응급실이 있어서 다행이네요. 큰 문제는 없으신 거죠?
5197,파편이 튀어서 그 때 저도 응급실 가서 치료 받기도 했고…,응급실에 가셨다니 정말 놀랐어요. 아무 문제 없으신가요? 걱정 되네요.
5213,지금 상태가 너무 안 좋아서 학교 안 나가고 있어요.,상태가 더 안 좋아지셨군요. 걱정이 되네요.
5214,진짜 심해진 거 같긴 해요.,정말 힘드시겠어요. 지금도 증상이 심하신가요?


## 2-2. 데이터 전처리

단어들을 따로 입력하기 위해  
물음표 마침표 등을 띄워주는 전처리 작업을 거쳤다.



In [16]:
def preprocess_sentence(sentence):
    sentence = sentence.strip() #양 옆의 공백 제거

  # 단어와 구두점(punctuation) 사이의 거리를 만듭니다.
  # 예를 들어서 "배고프다." => "배고프다 ."와 같이
  # 온점 사이에 거리를 만듭니다.
    sentence = re.sub(r"([?.!,-])", r" \1 ", sentence)

  # (a-z, A-Z, ".", "?", "!", ",")를 제외한 모든 문자를 공백인 ' '로 대체합니다.
  # sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence) 영어의 경우
    sentence = re.sub(r"([a-zA-Z]+)", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

  # sentence = re.sub(r"^[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", " ", sentence) #제대로 된 한국어가 아닌 경우 ' ' 로 대체합니다
    sentence = sentence.strip()
    
    return sentence

In [17]:
Q = list(map(preprocess_sentence, data_filter['유저']))
A = list(map(preprocess_sentence, data_filter['챗봇']))

print(f'전체 샘플 수, 질문 : {len(Q)} , 대답 : {len(A)}\n\n')

cnt = 0
for a, b in zip(Q,A):
    print(f'질문 : {a}')
    print(f'대답 : {b}')
    print()
    cnt += 1
    if cnt > 5:
          break

전체 샘플 수, 질문 : 1034 , 대답 : 1034


질문 : 제 감정이 이상해진 것 같아요 . 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요 .
대답 : 감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요 .

질문 : 더 이상 내 감정을 내가 컨트롤 못 하겠어 .
대답 : 저도 그 기분 이해해요 . 많이 힘드시죠 ?

질문 : 하루종일 오르락내리락 롤러코스터 타는 기분이에요 .
대답 : 그럴 때는 밥은 잘 먹었는지 , 잠은 잘 잤는지 체크해보는 것도 좋아요 .

질문 : 평소 다른 일을 할 때도 비슷해요 . 생각한대로 안되면 화가 나고…그런 상황이 지속되면 폭발해버려요 .
대답 : 화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요 .

질문 : 예전보다 화내는 게 과격해진 거 같아 .
대답 : 정말 힘드시겠어요 . 화는 남에게도 스스로에게도 상처를 주잖아요 .

질문 : 화가 안 참아져 .
대답 : 화가 너무 많이 날 때는 심호흡을 해보는 게 어떨까요 ? 씁 - 후 -



In [21]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(Q + A, target_vocab_size=2**13)


In [22]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

tokenizer.vocab_size

6358

In [23]:
print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

START_TOKEN의 번호 : [6358]
END_TOKEN의 번호 : [6359]


In [24]:
VOCAB_SIZE = tokenizer.vocab_size + 2 # start 토큰과 end 토큰을 추가하였다.
print(VOCAB_SIZE)

6360


In [25]:
# 임의의 22번째 샘플에 대해서 정수 인코딩 작업을 수행.
# 각 토큰을 고유한 정수로 변환

print(f'질문 : {Q[2]}')
print(f'대답 : {A[2]}\n')
    
print('정수 인코딩 후의 첫번째 질문 샘플: {}'.format(tokenizer.encode(Q[2])))
print('정수 인코딩 후의 첫번째 답변 샘플: {}'.format(tokenizer.encode(A[2])))

질문 : 하루종일 오르락내리락 롤러코스터 타는 기분이에요 .
대답 : 그럴 때는 밥은 잘 먹었는지 , 잠은 잘 잤는지 체크해보는 것도 좋아요 .

정수 인코딩 후의 첫번째 질문 샘플: [227, 3325, 4830, 779, 1781, 1]
정수 인코딩 후의 첫번째 답변 샘플: [34, 42, 4441, 20, 4681, 4, 604, 20, 2870, 2417, 23, 99, 1]


같은 마침표는 같은 숫자인 1 로 표기되는 것을 볼 수 있다.

## 2-3. 모델 학습

### 2-3-1. split

In [27]:
total_data_text = Q + A

total_data_text = [tokenizer.encode(text) for text in total_data_text]

# 텍스트데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)

# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,  
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 문장의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens) / len(num_tokens)))

문장길이 평균 :  9.458413926499032
문장길이 최대 :  25
문장길이 표준편차 :  3.63755819096933
pad_sequences maxlen :  16
전체 문장의 0.960348162475822%가 maxlen 설정값 이내에 포함됩니다. 


In [28]:
MAX_LENGTH = 16

In [29]:
# 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
def tokenize_and_filter(inputs, outputs):
    tokenized_inputs, tokenized_outputs = [], []
  
    for (sentence1, sentence2) in zip(inputs, outputs):
    # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
        sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
        sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    # 최대 길이 28 이하인 경우에만 데이터셋으로 허용
        if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
            tokenized_inputs.append(sentence1)
            tokenized_outputs.append(sentence2)
  
    # 최대 길이 28으로 모든 데이터셋을 패딩
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
    return tokenized_inputs, tokenized_outputs

In [30]:
questions, answers = tokenize_and_filter(Q,A)

print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 질문 샘플 개수: {}'.format(len(Q)))
print('필터링 후의 답변 샘플 개수: {}'.format(len(A)))

단어장의 크기 : 6360
필터링 후의 질문 샘플 개수: 1034
필터링 후의 답변 샘플 개수: 1034


In [32]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더는 이전의 target을 다음의 input으로 사용합니다.
# 이에 따라 outputs에서는 START_TOKEN을 제거하겠습니다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

### 2-3-2. model

In [33]:
# 포지셔널 인코딩 레이어
class PositionalEncoding(tf.keras.layers.Layer):

    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
    # 각도 배열 생성
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)

    # 배열의 짝수 인덱스에는 sin 함수 적용
        sines = tf.math.sin(angle_rads[:, 0::2])
        # 배열의 홀수 인덱스에는 cosine 함수 적용
        cosines = tf.math.cos(angle_rads[:, 1::2])

        # sin과 cosine이 교차되도록 재배열
        pos_encoding = tf.stack([sines, cosines], axis=0)
        pos_encoding = tf.transpose(pos_encoding,[1, 2, 0]) 
        pos_encoding = tf.reshape(pos_encoding, [position, d_model])

        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]


def scaled_dot_product_attention(query, key, value, mask):
      # 어텐션 가중치는 Q와 K의 닷 프로덕트
    matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 가중치를 정규화
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

  # 패딩에 마스크 추가
    if mask is not None:
        logits += (mask * -1e9)

      # softmax적용
        attention_weights = tf.nn.softmax(logits, axis=-1)

      # 최종 어텐션은 가중치와 V의 닷 프로덕트
        output = tf.matmul(attention_weights, value)
    return output


class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # Q, K, V에 각각 Dense를 적용합니다
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # 병렬 연산을 위한 머리를 여러 개 만듭니다
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # 스케일드 닷 프로덕트 어텐션 함수
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # 최종 결과에도 Dense를 한 번 더 적용합니다
        outputs = self.dense(concat_attention)

        return outputs


def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]


def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

# 인코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 두 개의 서브 레이어가 존재합니다.
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
        inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

      # 패딩 마스크 사용
        padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

      # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
        attention = MultiHeadAttention(
          d_model, num_heads, name="attention")({
              'query': inputs,
              'key': inputs,
              'value': inputs,
              'mask': padding_mask
          })

      # 어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행
        attention = tf.keras.layers.Dropout(rate=dropout)(attention)
        attention = tf.keras.layers.LayerNormalization(
          epsilon=1e-6)(inputs + attention)

      # 두 번째 서브 레이어 : 2개의 완전연결층
        outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
        outputs = tf.keras.layers.Dense(units=d_model)(outputs)

      # 완전연결층의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
        outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
        outputs = tf.keras.layers.LayerNormalization(
          epsilon=1e-6)(attention + outputs)

        return tf.keras.Model(
          inputs=[inputs, padding_mask], outputs=outputs, name=name)

def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 패딩 마스크 사용
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 임베딩 레이어
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # num_layers만큼 쌓아올린 인코더의 층.
    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(
    inputs=[inputs, padding_mask], outputs=outputs, name=name)



# 디코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 세 개의 서브 레이어가 존재합니다.
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(
        shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
    attention1 = MultiHeadAttention(
        d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
        })

  # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
    attention1 = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(attention1 + inputs)

  # 두 번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행 (인코더-디코더 어텐션)
    attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
        })

  # 마스크드 멀티 헤드 어텐션의 결과는
  # Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 세 번째 서브 레이어 : 2개의 완전연결층
    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')

  # 패딩 마스크
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  # 임베딩 레이어
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  # Dropout이라는 훈련을 돕는 테크닉을 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
          units=units,
          d_model=d_model,
          num_heads=num_heads,
          dropout=dropout,
          name='decoder_layer_{}'.format(i),
      )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name)


def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더에서 패딩을 위한 마스크

    enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용합니다.
  # 내부적으로 패딩 마스크도 포함되어져 있습니다.
    look_ahead_mask = tf.keras.layers.Lambda(
        create_look_ahead_mask,
        output_shape=(1, None, None),
        name='look_ahead_mask')(dec_inputs)

  # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
  # 디코더에서 패딩을 위한 마스크
    dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더
    enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
    )(inputs=[inputs, enc_padding_mask])

  # 디코더
    dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
    )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 완전연결층
    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [34]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 2 # 인코더와 디코더의 층의 개수
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기, 노드
DROPOUT = 0.1 # 드롭아웃의 비율

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None, None)]       0           []                               
                                                                                                  
 dec_inputs (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 enc_padding_mask (Lambda)      (None, 1, 1, None)   0           ['inputs[0][0]']                 
                                                                                                  
 encoder (Functional)           (None, None, 256)    2682368     ['inputs[0][0]',                 
                                                                  'enc_padding_mask[0][0

In [35]:
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
    loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32) #패딩마스크
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

In [36]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [39]:
#learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
     beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [40]:
EPOCHS = 20
model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/20
14/14 [==============================] - 17s 432ms/step - loss: 4.9961 - accuracy: 0.0616
Epoch 2/20
14/14 [==============================] - 6s 392ms/step - loss: 4.0996 - accuracy: 0.0824
Epoch 3/20
14/14 [==============================] - 5s 369ms/step - loss: 3.8326 - accuracy: 0.1325
Epoch 4/20
14/14 [==============================] - 5s 363ms/step - loss: 3.5484 - accuracy: 0.1576
Epoch 5/20
14/14 [==============================] - 5s 358ms/step - loss: 3.1805 - accuracy: 0.1925
Epoch 6/20
14/14 [==============================] - 5s 360ms/step - loss: 2.8175 - accuracy: 0.2247
Epoch 7/20
14/14 [==============================] - 5s 363ms/step - loss: 2.4900 - accuracy: 0.2624
Epoch 8/20
14/14 [==============================] - 5s 361ms/step - loss: 2.1783 - accuracy: 0.3078
Epoch 9/20
14/14 [==============================] - 5s 360ms/step - loss: 1.9014 - accuracy: 0.3556
Epoch 10/20
14/14 [==============================] - 5s 372ms/step - loss: 1.6237 - accuracy: 0.414

## 2-4. 데이터 평가

In [41]:
def decoder_inference(sentence):
    sentence = preprocess_sentence(sentence)

  # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
  # ex) Where have you been? → [[8331   86   30    5 1059    7 8332]]
    sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
  # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장. ex) 8331
    output_sequence = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 인퍼런스 단계
    for i in range(MAX_LENGTH):
    # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
        predictions = model(inputs=[sentence, output_sequence], training=False)
        predictions = predictions[:, -1:, :]

    # 현재 예측한 단어의 정수
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
        if tf.equal(predicted_id, END_TOKEN[0]):
            break

    # 예측한 단어들은 지속적으로 output_sequence에 추가됩니다.
    # 이 output_sequence는 다시 디코더의 입력이 됩니다.
        output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

    return tf.squeeze(output_sequence, axis=0)

In [42]:
def sentence_generation(sentence):
      # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
    prediction = decoder_inference(sentence)

  # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
    predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

    print('입력 : {}'.format(sentence))
    print('출력 : {}'.format(predicted_sentence))

    return predicted_sentence 

In [43]:
sentence_generation('나 너무 힘들어')


입력 : 나 너무 힘들어
출력 : 정말 당황스러우셨겠어요 . 하지만 너무 무리해서 생각해낼 필요는 없답니다 .


'정말 당황스러우셨겠어요 . 하지만 너무 무리해서 생각해낼 필요는 없답니다 .'

In [44]:
sentence_generation('부트캠프 안하고 싶어 집에 갈래')


입력 : 부트캠프 안하고 싶어 집에 갈래
출력 : 그 마음 다 이해해요 . 그럴 땐 맛있는 음식을 먹으면 조금 나아져요 .


'그 마음 다 이해해요 . 그럴 땐 맛있는 음식을 먹으면 조금 나아져요 .'

다만 현재 들어가있는 정보들은 모두  
위로해 주는 문답이기 때문에  
어떤 말을 해도 위로하는 답변밖에 나오지 않는다는 한계가 있다.

In [45]:
sentence_generation('나 오늘 행복해 사실')


입력 : 나 오늘 행복해 사실
출력 : 일이 많아 스트레스가 많겠어요 . 몸 상하지 않게 잘 챙겨 드세요 .


'일이 많아 스트레스가 많겠어요 . 몸 상하지 않게 잘 챙겨 드세요 .'

In [46]:
sentence_generation('나 스트레스 안많은디')


입력 : 나 스트레스 안많은디
출력 : 그 마음 다 이해해요 . 그럴 땐 맛있는 음식을 먹으면 조금 나아져요 .


'그 마음 다 이해해요 . 그럴 땐 맛있는 음식을 먹으면 조금 나아져요 .'

# 3. POTG
## 3-1. 소감

### "너무 부족해요,, 이제 좀 더 잘 해볼게요 🥶"
지식이 너무 짧음을 매일 매일 실감합니다.  
앞으로도 더 공부를 해야 할 것 같아요 ㅠㅠ

## 3-2. 어려웠던 점과 극복방안

1. 엑셀 파일을 열기 위해서는 'xlrd' 라는 라이브러리가 필요하다.  
쉽게 `pip install xlrd` 로 설치한 후 이용해 주었다.

2. SubwordTextEncoder API 를 사용하기 위해서  
 `tfds.deprecated.text` 라이브러리를 사용하려 했으나 계속 에러가 났다.  
 이유는 버전문제였고 2.3+ 이하 버전은 `tfds.features.text` 를 사용해야 한다.